In [1]:

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "5"
checkpoint = "mistralai/Mathstral-7B-v0.1"

/data03/sunyi/conda/envs/qwen25_math_eval/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", torch_dtype=torch.bfloat16)

Loading checkpoint shards: 100%|██████████| 6/6 [00:08<00:00,  1.34s/it]


In [19]:
system_prompt = "Please reason step by step, and put your final answer within \\boxed{{}}."
question = "A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?"
messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": question}]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
print(text)
print(type(text))
inputs = tokenizer([text], return_tensors="pt").to(model.device)
print(inputs)
print(tokenizer.chat_template)
# prompt = question + "\nPlease reason step by step, and put your final answer within \\boxed{{}}." + "1. The robe takes 2 bolts of blue fiber.\n2. It takes half as much white fiber as blue fiber." + "\n\nFinal answer within \\boxed{{}}:\n"
# inputs = tokenizer([prompt], return_tensors="pt").to(model.device)
# prompt = question + "Solve the task through two types of reasoning:\n1. Coarse-Grained Reasoning: give quick analysis step by step and an answer. Focus on efficiency and simplicity.\n2. Fine-Grained Reasoning: give detailed analysis step by step and a refined answer. Foucus on accuracy and correctness.\nPut final answer within \\boxed{{}}.\n\nOutput format:\n**Coarse Reasoning**\n\n**Fine Reasoning**\n\n**Final Answer** within \\boxed{{}}"
# inputs = tokenizer([prompt], return_tensors="pt").to(model.device)
# "To solve this problem, we start by understanding that the robe requires 2 bolts of blue fiber. The problem states that the robe also requires half as much white fiber as blue fiber."

<s>[INST] Please reason step by step, and put your final answer within \boxed{{}}.

A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?[/INST]
<class 'str'>
{'input_ids': tensor([[    1,     1,     3,  6687,  3379,  4475,  1254,  4475, 29493,  1072,
          2426,  1342,  2248,  5140,  3141,  1182,  3626,  1054,  7420,  1743,
         29491,   781,   781, 29509,  1480,  1873,  5115, 29473, 29518, 17129,
          1842,  1070,  5813, 19635,  1072,  3563,  1137,  1956,  3843, 19635,
         29491, 29473,  2370,  2055, 17129,  1842,  1065,  3870,  2003,  1146,
          2156, 29572,     4]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]], device='cuda:0')}
{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content'] %}
    {%- s

: 

In [14]:
out = model.generate(**inputs, max_new_tokens=512, do_sample=False)
print(out)
tokenizer.decode(out[0])

# >>> '<s>[INST] What are the roots of unity?[/INST] The roots of unity are the complex numbers that satisfy the equation $z^n = 1$, where $n$ is a positive integer. These roots are evenly spaced around the unit circle in the complex plane, and they have a variety of interesting properties and applications in mathematics and physics.</s>'


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


tensor([[    1,     1,     3,  6687,  3379,  4475,  1254,  4475, 29493,  1072,
          2426,  1342,  2248,  5140,  3141,  1182,  3626,  1054,  7420,  1743,
         29491,   781,   781, 29509,  1480,  1873,  5115, 29473, 29518, 17129,
          1842,  1070,  5813, 19635,  1072,  3563,  1137,  1956,  3843, 19635,
         29491, 29473,  2370,  2055, 17129,  1842,  1065,  3870,  2003,  1146,
          2156, 29572,     4,  2559, 12817,  1224,  3468, 29493,  1246,  1917,
          1254,  7167,  1137,  1040,  1480,  1873,  7716, 29473, 29518, 17129,
          1842,  1070,  5813, 19635, 29491,  1183,  3468,  5373,  1137,  1040,
          1480,  1873,  1603,  7716,  3563,  1158,  1956,  3843, 19635,  1158,
          5813, 19635, 29491,  9237, 29493,  1246, 14679,  1040,  4326,  1070,
          3843, 19635,  4004,  1158,  6872, 29515,   781,   781, 29501,  3508,
          1464,  1070,  3843, 19635,  1095,  1646,  2423, 29519, 29508,  2168,
         29518, 29520,  1182,  5361, 29473, 29518, 2

'<s><s>[INST] Please reason step by step, and put your final answer within \\boxed{{}}.\n\nA robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?[/INST] To solve this problem, we start by understanding that the robe requires 2 bolts of blue fiber. The problem states that the robe also requires half as much white fiber as blue fiber. Therefore, we calculate the amount of white fiber needed as follows:\n\n- Amount of white fiber = $\\frac{1}{2} \\times 2$ bolts of blue fiber\n- Amount of white fiber = $1$ bolt of blue fiber\n\nNow, to find the total amount of fiber needed for the robe, we add the bolts of blue fiber and the bolts of white fiber together:\n\n- Total bolts of fiber = Bolts of blue fiber + Bolts of white fiber\n- Total bolts of fiber = $2 + 1$\n- Total bolts of fiber = $3$\n\nTherefore, the total number of bolts of fiber needed for the robe is $\\boxed{3}$.</s>'